In [9]:
#============================================================================
# BM25 예제임
# => 
#============================================================================
# bm25 설치
# !pip install rank_bm25
import konlpy
from konlpy.tag import Mecab
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm


contexts = [
    "세계 배달 피자 리더 도미노피자가 우리 고구마를 활용한 신메뉴를 출시한다.도미노피자는 오는 2월 1일 국내산 고구마와 4가지 치즈가 어우러진 신메뉴 `우리 고구마 피자`를 출시하고 전 매장에서 판매를 시작한다. 이번에 도미노피자가 내놓은 신메뉴 `우리 고구마 피자`는 까다롭게 엄선한 국내산 고구마를 무스와 큐브 형태로 듬뿍 올리고, 모차렐라, 카망베르, 체더 치즈와 리코타 치즈 소스 등 4가지 치즈와 와규 크럼블을 더한 프리미엄 고구마 피자다.",
    "독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다.지원단에 따르면 오전 8시32분쯤 해경 1511함이 동체로부터 2.1㎞ 떨어진 지점에서 헬기 잔해물 1점을, 13분 뒤 해양 2000호가 동체로부터 4.7㎞ 떨어진 지점에서 2점을 발견했다.또 8시55분 해경 5001함이 동체로부터 4.1㎞ 떨어진 지점에서 1점을 발견했다.해양 2000호가 발견한 잔해물은 기체 창문으로 밝혀졌다. 지원단은 나머지 3점의 잔해물이 어떤 것인지 확인하고 있다. 지원단은 이날 현재 함선 15척과 항공기 6대를 동원해 실종자 수색작업을 계속하고 있다.청해진함과 광양함은 원격 무인잠수정(rov)을 이용해 오전 9시부터 미탐색 구역과 동체 발견지점 기준에서 남서쪽으로 확장된 구역에서 정밀탐색을 벌이고 있다.피항한 해군함 3척은 기상이 좋아지면 오후 1시30분쯤 현장에 복귀할 예정이다.또 수심 40ｍ 이내 독도 연안해역에는 잠수사를 37명을 투입하고 해안가에는 드론과 소형구조 보트, 독도경비대 등을 동원해 수색하고 있다.이같은 수색에도 당국은 지난 5일 오후 5시45분에 수습, 인양한 88대왕호 선원 고(故) (이름)씨(50) 이후에는 실종자를 발견하지 못하고 있다.닷새가 지나도록 실종자가 추가적으로 발견되지 않자 수색당국은 (이름) 국무총리가 언급한 민간 잠수사 투입 방안도 고려 중이지만 독도 해역의 기상 상태가 좋지 않아 투입 시기를 저울질하는 것으로 알려졌다.지난 9일 이 총리는 피해 가족들이 머무르는 대구 강서소방서 가족 대기실을 찾아 '독도 수역에 민간잠수사 투입 등을 (이름) 고려하고 모든 것을 원점에서 재검토하겠다'고 밝힌 바 있다.앞서 지난 10월 31일 오후 11시 25분쯤 응급환자와 보호자, 소방대원 5명 등 7명이 탄 중앙119구조본부 소속 ec225 헬기 한 대가 독도에서 이륙한 직후 바다로 떨어졌다.당시 소방대원들은 구급활동 임무를 수행하다 환자와 보호자 등을 헬기에 태우고 이륙 2~3분만에 헬기와 함께 바다로 추락했다.수색당국은 현재까지 독도 해역에서 (이름)(39) 부기장과 (이름)(45) 정비실장, 조업 중 손가락이 절단돼 이송되던 선원 (이름)씨(50) 등 시신 3구를 수습해 대구 계명대 동산병원에 안치했다.기장 (이름)씨(46), 구(이름) (이름)씨(29 여)와 배혁씨(31), 선원 박기동씨(46)의 생사는 현재까지 확인되지 않고 있다.",
    "도미노피자가 삼일절을 맞아 '방문포장 1+1' 이벤트를 진행한다. 이번 이벤트는 도미노피자 102개 매장에서 3월 1일 단 하루 동안 방문포장 온라인, 오프라인 주문 시 피자 1판을 더 증정하는 이벤트다. 온라인 주문 시 장바구니에 2판을 담은 후 할인 적용이 가능하며, 동일 가격 또는 낮은 가격의 피자를 고객이 선택하면 무료로 증정한다.",
    "정부가 독도 해역에서 추락한 헬기와 동일한 기종의 헬기 도입 방침 철회를 검토하기로 했다.진영 행정안전부 장관은 5일 사고 희생자 2명의 유족과 실종자 가족들이 머무르고 있는 대구 강서소방서를 찾아 '사고 헬기와 동일한 기종을 도입하는 방침을 철회하는 것을 검토 중'이라고 밝혔다.소방당국은 대당 460억원 상당의 28인용 수송헬기 2대를 내년부터 2023년쯤까지 순차적으로 도입할 계획을 세우고 예산 집행까지 완료한 것으로 전해졌다.소방당국이 도입하려는 헬기는 프랑스 에어버스헬리콥터스(옛 유로콥터)사의 'h225'로, 명칭만 다를 뿐 독도 해역에서 추락한 'ec225'와 같은 기종이다.h225 기종은 2016년 4월 노르웨이 해상을 지나던 중 주회전날개가 본체에서 떨어지면서 추락해 탑승자 13명이 숨지는 등 기체 안전성 논란이 많았다.피해 가족 등이 이날 도입 철회를 강하게 주장하자 진 장관은 '기체 결함을 보완한 기종을 도입하는 것으로 알고 있지만 피해 가족들의 요구대로 도입 철회를 검토하겠다. 향후 이런 사고가 발생하지 않도록 정부가 최선을 다 하겠다'고 약속했다.이날 독도 해역 소방헬기 추락 사고 희생자와 실종자 가족들은 '당국의 초동 대처와 이후 일련의 조치 상황이 미흡했다'며 정부에 강한 불만을 쏟아냈다.가족들은 특히 kbs가 독도 헬기 사고 관련 영상을 보유한 사실을 숨기고 경찰의 공유 요청을 거절한 의혹에 대해 해당 직원과 kbs 사장의 공식적인 사과를 촉구했다.희생자 2명의 유족과 실종자 가족들은 진 장관에게 '우리가 난리를 치니까 온 것 아니냐', '생명에 차별을 두지 말라', '국가의 일을 하다 순직했는데 장관이 모든 걸 놓치고 있다. 국민은 누구를 믿고 행동해야 하느냐', '사고 초동부터 지금까지의 수색 방침이 미흡했다'고 따졌다.진 장관이 유족과 실종자 가족들에게 '안타깝고, 죄송하다. 최선을 다해 수색에 전념하고 있다. 매일 상황 보고받고 있다'며 '불편한 점이 있다면 보완 지원하겠다. 해경에서 수색을 총괄하고 있으니 원한다면 과정과 그동안의 상황을 잘 설명하겠다'고 했지만 이들의 반발은 수그러들지 않았다.피해 가족들의 계속된 질문에 진 장관은 '정부가 최선을 다하고 있다'는 답변만 되풀이했다.이 사고로 생사가 확인되지 않은 소방대원 (이름)씨(29)의 어머니는 '이제는 눈물도 나지 않는다. 아무 것도 모르는 제 소견으로는 헬기가 잘못된 것 같다. 딸 아이의 죽음이 헛되지 않도록 소방에 힘을 보태달라'고 주문했다.다른 실종자의 친인척은 kbs 영상과 관련해 '만약 파일이 삭제됐다면 복원해달라. 아니면 해당 직원이 직접 와서 우리에게 추락 상황을 본 그대로 사실대로 말해달라. 언론 등에서 제기하는 수색당국의 공유 요청을 고의로 거부했다면 직원은 물론 kbs 사장도 우리 앞에 사과해야 한다'고 했다.진 장관 방문 이후 (이름) 대구시 행정부시장이 강서소방서를 방문하자 일부 가족들이 '피해자 가족이 대구에 머문지 이틀이나 지났는데 왜 이제야 찾았느냐'고 항의했다.한 관계자는 '내년 총선 출마설이 나도는 이 부시장이 행안부 장관이 오니까 눈도장을 찍으러 온 것 아니냐'는 말도 했다.한편 중앙119구조본부 소속 소방헬기 ec225는 지난달 31일 오후 11시25분쯤 독도경비대로부터 독도 남쪽 6해리(약 11㎞) 부근에서 어로 작업 도중 손가락이 절단된 어선 선원을 (이름)던 중 추락했다.같은날 긴급 환자 발생 신고를 접수한 후 오후 9시33분 대구에서 이륙한 소방헬기는 오후 11시23분쯤 독도에 도착해 환자를 태운 뒤 이륙 2~3분 만에 바다에 떨어진 것으로 알려졌다.사고 헬기에는 기장 (이름)씨(46), 부기장 (이름)씨(39), 정비실장 (이름)씨(45), 구(이름) (이름)씨(29 여), 배혁씨(31)와 88대왕호 선원 (이름)씨(50), 박기동씨(46) 등 7명이 탑승했다.이들 중 현재까지 부기장 이씨와 정비실장 서씨의 시신만 수습됐으며, 5일 0시38분쯤 수색당국이 동체 인양 위치와 동일한 위치에서 실종자로 추정되는 시신 1구를 추가로 발견했다.이 시신은 지난 3일 동체 인양 중 유실된 실종자로 추정되며 심해잠수부들의 포화잠수를 활용해 이날 중 수습이 가능할 것으로 예상된다.시신이 수습되면 대구 계명대 동산병원 장례식장에 안치될 예정이다.",
]

def tokenizer(sent):
  return sent.split(" ")

# 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
# Mecab 선언
mecab = Mecab()

mecab_contexts=[]
for context in tqdm(contexts):
    #temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
    temp = mecab.nouns(context)
    #print(temp)
    sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
    mecab_contexts.append(sentence)
    
print(mecab_contexts)
print(len(mecab_contexts))        


# tokeniaer 후 인덱싱
tokenized_corpus = [tokenizer(doc) for doc in mecab_contexts]
bm25 = BM25Okapi(tokenized_corpus)

print(bm25.doc_len)
print(bm25.doc_freqs)

  0%|          | 0/4 [00:00<?, ?it/s]

['세계 배달 피자 리더 도미노피자 우리 고구마 활용 메뉴 출시 도미노피자 월 일 국내산 고구마 가지 치즈 메뉴 우리 고구마 피자 출시 매장 판매 시작 이번 도미노피자 메뉴 우리 고구마 피자 엄선 국내 고구마 무스 큐브 형태 모차렐라 카망베르 체더 치즈 리코타 치즈 소스 등 가지 치즈 규 크 럼블 프리미엄 고구마 피자', '독도 해역 헬기 추락 사고 발생 지 열 하루 실종자 추가 발견 소식 헬기 동체 잔해 물 부유물 등 발견 정작 실종자 발견 수색 장기 것 우려 현실 조짐 실종자 가족 애 정부 현장 수습 원단 지 원단 일 오전 시 브리핑 이날 오전 독도 해역 수색 결과 점 부유물 추가 발견 인양 원단 오전 시 분 해경 함 동체 지점 헬기 잔해 물 점 분 뒤 해양 호 동체 지점 점 발견 시 분 해경 함 동체 지점 점 발견 해양 호 발견 잔해 물 기체 창문 원단 나머지 점 잔해 물 것 확인 원단 이날 함선 척 항공기 대 동원 실종자 수색 작업 계속 청해진 광 양 원격 무인 잠수정 이용 오전 시 미 탐색 구역 동체 발견 지점 기준 남서쪽 확장 구역 정밀 탐색 피 항한 해군 척 기상 오후 시 분 현장 복귀 예정 수심 독도 연안 해역 잠 수사 명 투입 해안가 드론 소형 구조 보트 독도 경비대 등 동원 수색 수색 당국 일 오후 시 분 수습 인양 대 왕호 선원 고 이름 씨 이후 실종자 발견 닷새 실종자 추가 발견 수색 당국 이름 국무총리 언급 민간 잠 수사 투입 방안 고려 중 독도 해역 기상 상태 투입 시기 저울질 것 일 총리 피해 가족 대구 강서 소방서 가족 대기실 독도 수역 민간 잠 수사 투입 등 이름 고려 것 원점 검토 바 월 일 오후 시 분 응급 환자 보호자 소방대원 명 등 명 중앙 구조 본부 소속 헬기 대 독도 이 륙 직후 바다 당시 소방대원 구급 활동 임무 수행 환자 보호자 등 헬기 이 륙 분 만 헬기 바다 추락 수색 당국 현재 독도 해역 이름 부기장 이름 정비 실장 조업 중 손가락 절단 이송 선원 이름 씨 등 시신 구 수습 대구 계명 동산 병원 안치 기장 이름

In [7]:
#idf : 토큰의 inverse term frequency를 계산해둠
bm25.idf

{'세계': 0.5108256237659907,
 '배달': 0.5108256237659907,
 '피자': 0.11306437890464542,
 '리더': 0.5108256237659907,
 '도미노피자': 0.11306437890464542,
 '우리': 0.5108256237659907,
 '고구마': 0.5108256237659907,
 '활용': 0.5108256237659907,
 '메뉴': 0.5108256237659907,
 '출시': 0.5108256237659907,
 '월': 0.11306437890464542,
 '일': 0.11306437890464542,
 '국내산': 0.5108256237659907,
 '가지': 0.5108256237659907,
 '치즈': 0.5108256237659907,
 '매장': 0.11306437890464542,
 '판매': 0.5108256237659907,
 '시작': 0.5108256237659907,
 '이번': 0.11306437890464542,
 '엄선': 0.5108256237659907,
 '국내': 0.5108256237659907,
 '무스': 0.5108256237659907,
 '큐브': 0.5108256237659907,
 '형태': 0.5108256237659907,
 '모차렐라': 0.5108256237659907,
 '카망베르': 0.5108256237659907,
 '체더': 0.5108256237659907,
 '리코타': 0.5108256237659907,
 '소스': 0.5108256237659907,
 '등': 0.11306437890464542,
 '규': 0.5108256237659907,
 '크': 0.5108256237659907,
 '럼블': 0.5108256237659907,
 '프리미엄': 0.5108256237659907,
 '독도': 0.5108256237659907,
 '해역': 0.5108256237659907,
 '헬기': 0.51082

In [4]:
query = "고구마 피자 신메뉴"
tokenized_query = tokenizer(query)

# 쿼리에 따른 스코어 구함
doc_scores = bm25.get_scores(tokenized_query)
doc_scores

array([1.1424712 , 0.21246992, 0.14240611])

In [5]:
# 쿼리 문장 구함
bm25.get_top_n(tokenized_query, mecab_contexts, n=2)

['세계 배달 피자 리더 도미노피자 우리 고구마 활용 메뉴 출시 도미노피자 월 일 국내산 고구마 가지 치즈 메뉴 우리 고구마 피자 출시 매장 판매 시작 이번 도미노피자 메뉴 우리 고구마 피자 엄선 국내 고구마 무스 큐브 형태 모차렐라 카망베르 체더 치즈 리코타 치즈 소스 등 가지 치즈 규 크 럼블 프리미엄 고구마 피자',
 '피자 발상지 원조 수 남부 나폴리 식 피자 재료 본연 맛 집중 식감 자랑 대표 나폴리 피자 피자 마리 나라 피자 마르게리타']